In [1]:
from src.models.const_tree import C_Tree
from src.encs.constituent import *
from src.utils.constants import *
from src.models.linearized_tree import LinearizedTree
from nltk.tree import Tree

/home/poli/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# c_encs    = [C_ABSOLUTE_ENCODING, C_RELATIVE_ENCODING, C_INCREMENTAL_ENCODING, C_DYNAMIC_ENCODING]
# for enc in c_encs:
#     pred_const = "./test/pred.const."+enc+".labels"
#     pred_const_dec = "./test/pred.const."+enc+".decoded.trees"

#     decode_constituent(in_path = pred_const, out_path = pred_const_dec,
#                         encoding_type = enc, separator = "_", unary_joiner = "+", nulls = True,
#                         conflicts = C_STRAT_MAX, postags = False, lang = "en")

#     for line in open(pred_const_dec):
#         line = line.rstrip()
#         ct = C_Tree.from_string(line)
#         bt = C_Tree.to_binary(ct)
#         dt = C_Tree.restore_from_binary(bt)
#         assert ct == dt
#     os.remove(pred_const_dec)


In [3]:
cts = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (ADJP (NNP Black)) (NP (NNP Monday))) (. .))"

ot = C_Tree.from_string(cts)
bt = C_Tree.to_binary(ot)
dt = C_Tree.restore_from_binary(bt)

print(ot==dt)

True


In [4]:
labels_str = "-BOS-	-BOS-	-BOS-"+'\n'+\
"``	``	1_S"+'\n'+\
"Managed	VBD	3_S+VP"+'\n'+\
"properly	RB	3_S+VP_ADVP"+'\n'+\
",	,	2_PP"+'\n'+\
"and	CC	2_PP"+'\n'+\
"with	IN	4_PP"+'\n'+\
"a	DT	5_NP"+'\n'+\
"long-term	JJ	5_NP"+'\n'+\
"outlook	NN	1_S"+'\n'+\
",	,	1_S"+'\n'+\
"these	DT	1_S_NP"+'\n'+\
"can	MD	2_VP"+'\n'+\
"become	VB	3_VP"+'\n'+\
"investment-grade	JJ	4_NP"+'\n'+\
"quality	NN	4_NP"+'\n'+\
"properties	NNS	1_S"+'\n'+\
".	.	1_S"+'\n'+\
"-EOS-	-EOS-	-EOS-"

lt = LinearizedTree.from_string(labels_str, mode="CONST")
enc = C_NaiveAbsoluteEncoding(separator="_", unary_joiner="+")
ct = enc.decode(lt)

# Get original tree
ot = ct.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
nltk_t = Tree.fromstring(str(ot))
nltk_t.pretty_print()

# Encode it as binary
bt   = C_Tree.to_binary(ot)
nltk_t = Tree.fromstring(str(bt))
nltk_t.pretty_print()

# Collapse unary nodes
cbt  = bt.collapse_unary()
nltk_t = Tree.fromstring(str(cbt))
nltk_t.pretty_print()

# Add postags
postags = cbt.get_postags()
lcbt = cbt.remove_preterminals()
nltk_t = Tree.fromstring(str(lcbt))
nltk_t.pretty_print()

# Add a dummy root
lcbt.add_root_node()
nltk_t = Tree.fromstring(str(lcbt))
nltk_t.pretty_print()

# Decode it from binary
dt = C_Tree.restore_from_binary(bt)

print(ot==dt)

                                                       S                                                                    
  _____________________________________________________|__________________________________________________________________   
 |                        S                                  |    |                     VP                                | 
 |              __________|____________                      |    |     ________________|__________                       |  
 |             VP         |            PP                    |    |    |                           VP                     | 
 |      _______|______    |    ________|______               |    |    |     ______________________|_____                 |  
 |     |      ADVP    |   |   |               NP             |    NP   |    |                            NP               | 
 |     |       |      |   |   |     __________|________      |    |    |    |            ________________|________        

In [8]:
def traverse_childs(t):
    labels = []
    for node in t.children:
        if node.is_terminal():
            label_string = ""
            if node.is_right_child():
                label_string+="l"
                if node.parent.is_right_child():
                    label_string+="L"
                elif node.parent.is_left_child():
                    label_string+="R"
    
            elif node.is_left_child():
                label_string+="r"
                if node.parent.is_right_child():
                    label_string+="L"
                elif node.parent.is_left_child():
                    label_string+="R"
            labels.append(label_string)
        
        child_labels = traverse_childs(node)
        for label in child_labels if child_labels != [] else []:
            labels.append(label)
    return labels

# Como se saca el valor de los no terminales? El 'parent'?
labels = traverse_childs(lcbt)
print(labels)

['rL', 'rR', 'rL', 'lL', 'rL', 'rL', 'rL', 'rL', 'lL', 'rL', 'rL', 'rR', 'rL', 'rL', 'rL', 'lL', 'lL']
